<a href="https://colab.research.google.com/github/richardnoack/stats-dinglery/blob/main/ASA_xG_ETL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
###Handling authentication with GCS

from google.colab import auth
auth.authenticate_user()
print('party on wayne')

from google.cloud import bigquery
project_id = 'mls-stats'
bq_client = bigquery.Client(project=project_id)


###loading up ASA things

from itscalledsoccer.client import AmericanSoccerAnalysis 
asa_client = AmericanSoccerAnalysis() 

### making tables look nicer
%load_ext google.colab.data_table


Gathering all players
Gathering all teams
Gathering all stadia
Gathering all managers
Gathering all referees
Finished initializing client
party on wayne


In [ ]:
#dir(AmericanSoccerAnalysis)
### importing all the ASA tables needed
mls_players = asa_client.get_players(leagues="mls").drop('season_name', axis = 1)

player_xg = asa_client.get_player_xgoals(leagues = 'mls').explode('team_id')#.drop('team_id', axis= 1) ### this combines stats across all teams a player played for

mls_teams = asa_client.get_teams(leagues = 'mls')

team_xg = asa_client.get_team_xgoals(leagues= 'mls')

game_xg = asa_client.get_game_xgoals(leagues = 'mls')

#games = asa_client.get_games(leagues = 'mls')




In [ ]:
###running a test query
#bq_client.query('''SELECT * FROM mls-stats.asa_xg.rnoack_testing''').to_dataframe()
#### loading the first table back into bigquery
### use pandas gbq if needed https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_gbq.html#pandas.DataFrame.to_gbq

job_config = bigquery.LoadJobConfig(write_disposition = 'WRITE_TRUNCATE')

player_job = bq_client.load_table_from_dataframe(
    mls_players, 'mls-stats.asa_xg.mls_players', job_config=job_config
)  
#####
xg_game_job = bq_client.load_table_from_dataframe(
    player_xg, 'mls-stats.asa_xg.mls_player_xg', job_config=job_config
) 
#####
team_job = bq_client.load_table_from_dataframe(
    mls_teams, 'mls-stats.asa_xg.mls_teams', job_config=job_config
) 
#####
game_xg_job = bq_client.load_table_from_dataframe(
    game_xg, 'mls-stats.asa_xg.game_xg', job_config=job_config
) 
#####
team_xg_job = bq_client.load_table_from_dataframe(
    team_xg, 'mls-stats.asa_xg.team_xg', job_config=job_config
) 

team_xg_job.result()
game_xg_job.result()
player_job.result()
xg_game_job.result()
team_job.result()

/usr/local/lib/python3.7/dist-packages/google/cloud/bigquery/_pandas_helpers.py:275: UserWarning: Unable to determine type of column 'player_id'.
  warnings.warn(u"Unable to determine type of column '{}'.".format(column))
/usr/local/lib/python3.7/dist-packages/google/cloud/bigquery/_pandas_helpers.py:275: UserWarning: Unable to determine type of column 'team_id'.
  warnings.warn(u"Unable to determine type of column '{}'.".format(column))
/usr/local/lib/python3.7/dist-packages/google/cloud/bigquery/_pandas_helpers.py:275: UserWarning: Unable to determine type of column 'game_id'.
  warnings.warn(u"Unable to determine type of column '{}'.".format(column))


In [ ]:
#### importing the MLS schedule and writing it back to BQ
import pandas as pd 
json_schedule = pd.read_json('https://fgp-data-us.s3.amazonaws.com/json/mls_mls/rounds.json')
job_config = bigquery.LoadJobConfig(write_disposition = 'WRITE_TRUNCATE')

###https://medium.com/swlh/converting-nested-json-structures-to-pandas-dataframes-e8106c59976e
scheddf = json_normalize(json_schedule['matches'])
scheduledataframe = pd.DataFrame()
for i in scheddf:
  scheduledataframe = scheduledataframe.append(json_normalize(scheddf[i]))


#removing null rows
scheduledataframe = scheduledataframe.dropna(subset = ['home_squad_name'])
print(scheduledataframe.shape) ### --> (29, 25)


schedule_job = bq_client.load_table_from_dataframe(
    scheduledataframe, 'mls-stats.asa_xg.schedule', job_config=job_config
) 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/google/cloud/bigquery/_pandas_helpers.py:275: UserWarning: Unable to determine type of column 'home_squad_odds_id'.
  warnings.warn(u"Unable to determine type of column '{}'.".format(column))


(725, 26)
